In [1]:
from typing import Any, Optional, Union

import numpy as np
import pandas as pd
import simpy
import simpy.resources
from simpy.resources.store import StoreGet, StorePut

In [47]:
def create_grain_size_info():
    columns = ["group", "sub group","name", "code", "size lower [mm]", "size upper [mm]"]
    rows = [
      ["Very coarse soil", "Very coarse soil", "Large boulder",  "lBo",  630, np.inf],
      ["Very coarse soil", "Very coarse soil", "Boulder",  "Bo",  200, 630],
      ["Very coarse soil", "Very coarse soil", "Cobble",  "Co", 63, 200],
      ["Coarse soil",  "Gravel", "Coarse gravel", "cGr", 20, 63],
      ["Coarse soil",  "Gravel", "Medium gravel",  "mGr", 6.3, 20],
      ["Coarse soil",  "Gravel", "Fine gravel",  "fGr", 2.0, 6.3],
      ["Coarse soil",  "Sand",  "Coarse sand", "cSa", 0.63, 2.0],
      ["Coarse soil",  "Sand",  "Medium sand", "mSa", 0.2, 0.63],
      ["Coarse soil",  "Sand",  "Fine sand", "fSa", 0.063, 0.2],
      ["Fine soil", "Silt", "Coarse silt", "cSi", 0.02, 0.063],
      ["Fine soil", "Silt", "Medium silt", "mSi", 0.0063, 0.02],
      ["Fine soil", "Silt", "Fine silt",  "fSi", 0.002, 0.0063],
      ["Fine soil", "Clay", "Clay", "Cl", 0, 0.002]
    ]
    grain_sizes = pd.DataFrame(rows, columns=columns)
    def add_interval(df):
        interval = df.apply(
            lambda row: pd.Interval(
                row["size lower [mm]"], 
                row["size upper [mm]"], 
                closed="left"
            ),
            axis=1
        )
        df['interval'] = interval
        df = df.sort_values("size lower [mm]", ascending=True)
        return df
        
    agg = {"size lower [mm]": min, "size upper [mm]": max}
    grain_sizes_name = add_interval(grain_sizes.groupby('name').agg(agg))
    grain_sizes_code = add_interval(grain_sizes.groupby('code').agg(agg))
    grain_sizes_group = grain_sizes.groupby('group').agg(agg)
    grain_sizes_group = add_interval(grain_sizes_group)
    grain_sizes_sub_group = grain_sizes.groupby('sub group').agg(agg)
    grain_sizes_sub_group = add_interval(grain_sizes_sub_group)
    grain_sizes = {
        "grain_sizes_name": grain_sizes_name,
        "grain_sizes_code": grain_sizes_code,
        "grain_sizes_group": grain_sizes_group,
        "grain_sizes_sub_group": grain_sizes_sub_group
    }
    return grain_sizes


In [190]:

env = simpy.Environment()
store = simpy.Store(env=env)
layer = "Sediment layer A"
store.put(layer)
print(f"Store now has the following layers: {store.items}")
layer = "Sediment layer B"
store.put(layer)
print(f"Store now has the following layers: {store.items}")
layer = store.get()
print(f"Store now has the following layers: {store.items}")

Store now has the following layers: ['Sediment layer A']
Store now has the following layers: ['Sediment layer A', 'Sediment layer B']
Store now has the following layers: ['Sediment layer B']


Signature: resource.put(item: Any)
Docstring:
Request to put *item* into the *store*. The request is triggered once
there is space for the item in the store.
Type:      method


In [89]:
import dataclasses

@dataclasses.dataclass(init=True)
class Soil:
    volume_ratios: dict

@dataclasses.dataclass(init=True)
class SoilLayer:
    soil: Soil
    volume: float

In [187]:
class SedimentStoreGet(simpy.resources.store.StoreGet):
    """Request to get a *layer* from the *store* . The
    request is triggered once the soil is available in the store.

    *layer* is a SoilLayer
    :class:`StoreGet`.

    """

    def __init__(
        self,
        resource: 'SedimentStore',
        volume: float
    ):
        """An event to get a layer of sediment out of a sediment store."""
        self.volume = volume
        super().__init__(resource)

class SedimentStore(simpy.Store):
    def __init__(self, env: simpy.Environment, capacity: Union[float, int] = float('inf'), layers: list = None):
        super().__init__(env, capacity)
        # create a dictionary with info on grain sizes
        self._grain_size_info = create_grain_sizes()

    def get(
        self, 
        volume: float
    ) -> SedimentStoreGet:
        """Request to get a volume out of the sediment store."""
        return SedimentStoreGet(self, volume)

    
    def put(  # type: ignore[override] # noqa: F821
        self, item: Any, 
    ) -> StorePut:
        """Request to put *item* into the store. Item is """
        return StorePut(self, item)
    def _do_put(self, event: StorePut) -> Optional[bool]:
        # remaining capacity should allow for volume to fit in
        if self._capacity - self.level >= event.item.volume:
            self.items.append(event.item)
            event.succeed()
            return True
        return None
    def _do_get(self, event: SedimentStoreGet) -> Optional[bool]:
        print(f"getting {event.volume}")
        if self.level >= event.volume:
            # get a volume from the storage and return a new layer
            # update the current layers with the new layers
            new_items, layer = get_layer_from_top(layers=self.items, volume=event.volume)
            self.items = new_items
            event.succeed(layer)
        return None

    @property
    def grain_sizes(self):
        return self._grain_size_info["grain_sizes_code"]
        
    @property
    def level(self):
        return sum([layer.volume for layer in self.items]) 



import dataclasses

@dataclasses.dataclass(init=True)
class Soil:
    volume_ratios: dict

@dataclasses.dataclass(init=True)
class SoilLayer:
    soil: Soil
    volume: float



In [188]:

env = simpy.Environment()
sediment_store = SedimentStore(env=env)
sediment_store.grain_sizes

,size lower [mm],size upper [mm],interval
code,,,
Cl,0.0000,0.0020,"[0.0, 0.002)"
fSi,0.0020,0.0063,"[0.002, 0.0063)"
mSi,0.0063,0.0200,"[0.0063, 0.02)"
cSi,0.0200,0.0630,"[0.02, 0.063)"
fSa,0.0630,0.2000,"[0.063, 0.2)"
mSa,0.2000,0.6300,"[0.2, 0.63)"
cSa,0.6300,2.0000,"[0.63, 2.0)"
fGr,2.0000,6.3000,"[2.0, 6.3)"
mGr,6.3000,20.0000,"[6.3, 20.0)"


In [189]:
env = simpy.Environment()

sediment_store = SedimentStore(env=env, capacity=4)
soil = Soil(volume_ratios={"Cl": 0.9, "fSi": 0.1})
layer = SoilLayer(soil=soil, volume=2)
sediment_store.put(layer)
soil = Soil(volume_ratios={"Bo": 0.9, "Co": 0.1})
layer = SoilLayer(soil=soil, volume=2)
sediment_store.put(layer)
sediment_store.items

# this should not trigger because container is full
soil = Soil(volume_ratios={"cGr": 0.9, "Co": 0.1})
layer = SoilLayer(soil=soil, volume=2)
put_event = sediment_store.put(layer)
# should be one event in the queue
sediment_store.put_queue

sediment_store.get(volume=3)

sediment_store.items
env.step()
env.step()
env.step()
env.step()

sediment_store.put_queue
sediment_store.items






getting 3


[SoilLayer(soil=Soil(volume_ratios={'Bo': 0.9, 'Co': 0.1}), volume=2),
 SoilLayer(soil=Soil(volume_ratios={'cGr': 0.9, 'Co': 0.1}), volume=2)]

# Data types for sediment 
There are a few common methods to store an amount of sediment of different classes. Suppose we have 1kg of sediment with grain size 0.01mm and 2kg of sediment with size 0.02mm. Then we can store it as follows:
```
bins = [0.01, 0.02]
amount = [1, 2]
sediment_amount = (amount, bins)
```

Another way is to store the edges of the bins:
```
edges = [0.005, 0.015, 0.025]
hist = [1, 2]
sediment_histogram = (hist, edges)
```

You can consider the first example as a bar chart. You store the bins, but do not consider them on a continuous (interval, ratio), but rather as a nominal or ordinal variable. Using the edges approach you can also combine bins to make new bins, but you will lose information. Here we refer to the first data model as "bars" and the second as "histograms". 

``` 
sediment_a_bar = ([1, 2], [0.01, 0.02])
sediment_b_bar = ([1, 1], [0.01, 0.03])
mix_bars(sediment_a_bar, sediment_b_bar)
([2, 2, 1], (0.01, 0.02, 0.03)
```

``` 
sediment_a_hist = ([1, 2], [0.005, 0.015, 0.025])
sediment_b_hist = ([1, 0, 1], [0.005, 0.015, 0.025, 0.035])
mix_histograms(sediment_a, sediment_b)
([2., 2., 1.], [0.005, 0.015, 0.025, 0.035])
```

Besides mixing of sediment, we can also stack sediment. To do this we keep a list of histograms or bars. Using this list we can add and remove amounts of sediment. Based on the stacking (lifo) we can retrieve an amount of sediment and retrieve either bars or a histogram of sediment.

```
# a on top of b
sediment_stack = [
([1, 1], [0.01, 0.03]),
([1, 2], [0.01, 0.02])
]
get_bar(sediment_stack, 1)
([0.333, 0.667], [0.01, 0.02])
get_bar(sediment_stack, 2)
([0.667, 1.333], [0.01, 0.02])
```

Or from the histogram:
```
sediment_stack = [
    ([1, 0, 1], [0.005, 0.015, 0.025, 0.035]),
    ([1, 2], [0.005, 0.015, 0.025])
]
get_hist(sediment_stack, 1)
([0.005, 0.015, 0.025, 0.035], [0.01, 0.02, 0.0])
get_hist(sediment_stack, 2)
([0.005, 0.015, 0.025, 0.035], [0.01, 0.02, 0.0])

```


In [3]:
def mix_histograms(a, b):
    # from: https://stackoverflow.com/questions/47085662/merge-histograms-with-different-ranges
    if a is None:
        return b
    hist_a, edges_a = a
    if hist_a is None or edges_a is None:
        return b
    hist_b, edges_b = b
    d_a = np.diff(edges_a)[0]
    d_b = np.diff(edges_b)[0]
    d_int = np.min([d_a, d_b])

    min = np.min(np.hstack([edges_a, edges_b]))
    max = np.max(np.hstack([edges_a, edges_b]))
    # new edges
    edges_c = np.arange(min, max, d_int)

    def interpolate_hist(edges, hist, edges_int):
        # interpolate edges with counts (hist) to new edges (edges_int)
        cum_hist = np.hstack([0, np.cumsum(hist)])
        cum_hist_int = np.interp(edges_int, edges, cum_hist)
        hist_int = np.diff(cum_hist_int)
        return hist_int

    hist_a_int = interpolate_hist(edges_a, hist_a, edges_c)
    hist_b_int = interpolate_hist(edges_b, hist_b, edges_c)

    hist_c = hist_a_int + hist_b_int
    return hist_c, edges_c

class SedimentStore(simpy.Store):
    def __init__(self, env: simpy.Environment, capacity: Union[float, int] = float('inf')):
        super().__init__(env, capacity)
        # initialize with None
        self.hist = None
        self.bins = None
    def put(  # type: ignore[override] # noqa: F821
        self, item: Any, 
    ) -> StorePut:
        """Request to put *item* into the store. Item is """
        return StorePut(self, item)
    def _do_put(self, event: StorePut) -> Optional[bool]:
        print('putting', event)
        mix_histograms(self.hist, event.item)
        return None

    def _do_get(self, event: StoreGet) -> Optional[bool]:
        print('getting', event)
        if self.items:
            event.succeed(self.items.pop(0))
        return None

In [4]:
env = simpy.Environment()
store = SedimentStore(env)

In [5]:
put = store.put('hi')
put

putting <StorePut() object at 0x10e859820>


<StorePut() object at 0x10e859820>

In [6]:
env.run()

In [7]:
import numpy as np

In [8]:
a = np.histogram([0, 1, 1.2])
b = np.histogram([0, 1, 1.3])
mix_histograms(a, b)

(array([1.92307692, 0.07692308, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.38461538, 1.61538462, 1.23076923]),
 array([0.  , 0.12, 0.24, 0.36, 0.48, 0.6 , 0.72, 0.84, 0.96, 1.08, 1.2 ]))

In [9]:
np.max(np.hstack([a[1], b[1]]))

1.3

In [10]:

sediment_a = ([1, 2], [0.005, 0.015, 0.025])
sediment_b = ([1, 0, 1], [0.005, 0.015, 0.025, 0.035])
mix_histograms(sediment_a, sediment_b)

(array([2., 2., 1.]), array([0.005, 0.015, 0.025, 0.035]))